# Local Chatbot

This is the first version of a chatbot built entirely with open-source solutions, running on a simple laptop equipped with an NVIDIA GTX 4060.

It does not require an internet connection to run (only for downloading the Google Gemma model from Hugging Face).

As this is the first version, it still needs fine-tuning to learn specific domain knowledge.

The backend uses a ClickHouse columnar database, which allows it to query millions of rows at high speed.

A front-end interface and additional improvements are coming soon.

Information about mexican forest cover surface taken from the INEGI and https://databosques.cnf.gob.mx/

In [ ]:
import os
from huggingface_hub import login
import torch
from transformers import AutoTokenizer, BitsAndBytesConfig, GemmaTokenizer
from transformers.models.gemma3 import Gemma3ForCausalLM

my_token = "hf_abdcefghijklmnopqrstuvwxyz" #you need a huggingface token to download the model

login(token=my_token)

model_id = 'google/gemma-3-1b-it'
tokenizer = AutoTokenizer.from_pretrained(model_id, token=my_token)
model = Gemma3ForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16, device_map="auto", token=my_token, attn_implementation='eager')
# Set up the chat format
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.chat_template = "{{ bos_token }}{% if messages[0]['role'] == 'system' %}{{ raise_exception('System role not supported') }}{% endif %}{% for message in messages %}{% if (message['role'] == 'user') != (loop.index0 % 2 == 0) %}{{ raise_exception('Conversation roles must alternate user/assistant/user/assistant/...') }}{% endif %}{% if (message['role'] == 'assistant') %}{% set role = 'model' %}{% else %}{% set role = message['role'] %}{% endif %}{{ '<start_of_turn>' + role + '\n' + message['content'] | trim + '<end_of_turn>\n' }}{% endfor %}{% if add_generation_prompt %}{{'<start_of_turn>model\n'}}{% endif %}"


In [ ]:
#connection to clickhouse database (configured previously, instructions on the github of this same project)
import clickhouse_connect

client = clickhouse_connect.get_client(
    host='clickhouse',
    port=8123,  # HTTP interface
    username='your_user',
    password='your_password'
)

### Prompt context:

In [ ]:
sql_sintax_differences = '''
Relevant standard SQL vs ClickHouse SQL differences:

Basic Query Structure:

Use PREWHERE instead of WHERE for filtering on indexed columns for better performance
Standard: SELECT * FROM table WHERE condition
ClickHouse: SELECT * FROM table PREWHERE condition WHERE additional_condition

Limiting Results:

Use LIMIT BY instead of window functions for top N per group
Standard: SELECT * FROM (SELECT *, ROW_NUMBER() OVER (PARTITION BY category ORDER BY value DESC) as rn FROM table) WHERE rn <= 3
ClickHouse: SELECT * FROM table ORDER BY value DESC LIMIT 3 BY category

Top/Bottom Records:

Use argMax(column, order_by) instead of subqueries for getting values from max/min rows
Standard: SELECT name FROM table WHERE value = (SELECT MAX(value) FROM table)
ClickHouse: SELECT argMax(name, value) FROM table

Column Operations:

Use divide(a, b) or intDiv(a, b) for safer division
Standard: SELECT column1 / column2 FROM table
ClickHouse: SELECT divide(column1, column2) FROM table (handles division by zero)

String Functions:

Use position(haystack, needle) instead of LOCATE/CHARINDEX
Standard: LOCATE('substring', column)
ClickHouse: position(column, 'substring')

Aggregations:

Use uniq() instead of COUNT(DISTINCT)
Standard: SELECT COUNT(DISTINCT column) FROM table
ClickHouse: SELECT uniq(column) FROM table
Use groupArray() instead of string aggregation functions
Standard: GROUP_CONCAT(column) or STRING_AGG(column, ',')
ClickHouse: SELECT groupArray(column) FROM table GROUP BY category

Date Functions:

Use toStartOfMonth(), toStartOfWeek() instead of date truncation
Standard: DATE_TRUNC('month', date_column)
ClickHouse: toStartOfMonth(date_column)
Use dateDiff('day', date1, date2) with explicit time unit
Standard: DATEDIFF(date1, date2) (varies by database)
ClickHouse: dateDiff('day', date1, date2)

Conditional Logic:

Use multiIf() instead of nested CASE statements
Standard: CASE WHEN a > 0 THEN 'positive' WHEN a < 0 THEN 'negative' ELSE 'zero' END
ClickHouse: multiIf(a > 0, 'positive', a < 0, 'negative', 'zero')

Percentiles:

Use quantile(0.5)(column) instead of PERCENTILE_CONT
Standard: PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY column)
ClickHouse: SELECT quantile(0.5)(column) FROM table

Array Operations:

Use arrayJoin() to unnest arrays instead of UNNEST
Standard: SELECT * FROM table CROSS JOIN UNNEST(array_column)
ClickHouse: SELECT * FROM table ARRAY JOIN array_column

Ordering:

ClickHouse supports ORDER BY with NULLS FIRST/LAST like standard SQL
No difference needed here

Basic Math:

Use standard operators: +, -, *, /
Use round(column, digits) for rounding
Use abs(column) for absolute values
Same as standard SQL

Type Conversion:

Use toString(column) instead of CAST(column AS VARCHAR)
Use toInt32(column) instead of CAST(column AS INTEGER)
ClickHouse: SELECT toString(number_column), toInt32(string_column)

Sampling:

Use SAMPLE 0.1 for 10% sampling instead of TABLESAMPLE
Standard: SELECT * FROM table TABLESAMPLE(10 PERCENT)
ClickHouse: SELECT * FROM table SAMPLE 0.1

Format Output:

Use format(pattern, args...) for string formatting
Standard: varies by database (CONCAT, ||, +)
ClickHouse: format('Value: {}', column)
 '''

In [ ]:
schema_info = """Table in superficie_bd database:
    superficie_bd.superficie_forestal (
    entidad_federativa String,
    poblacion_2020 Float64,
    superficie_entidad_hectareas Float64,
    total_superficie_forestal_ha Float64,
    superficie_total_de_bosque_ha Float64,
    superficie_total_de_selva_ha Float64,
    superficie_total_de_manglar_ha Float64,
    superficie_otras_asociaciones_ha Float64,
    superficie_subtotal_arbolado_ha Float64,
    superficie_total_matorral_xerofilo_ha Float64,
    superficie_total_otras_areas_forestales_ha Float64,
    area_no_forestal_agricultura_de_humedad_ha Float64,
    area_no_forestal_agricultura_de_riego_ha Float64,
    area_no_forestal_agricultura_de_temporal_ha Float64,
    area_no_forestal_cuerpo_de_agua_ha Float64,
    area_no_forestal_acuicola_ha Float64,
    area_no_forestal_pastizal_cultivado_ha Float64,
    area_no_forestal_pastizal_inducido_ha Float64,
    area_no_forestal_sin_vegetacion_aparente_ha Float64,
    area_no_forestal_desprovisto_de_vegetacion_ha Float64,
    area_no_forestal_asentamientos_humanos_ha Float64,
    total_unidades_de_produccion_forestal UInt32,
    total_hectareas_produccion_forestal Float64,
    total_unidades_con_superficie_reforestada UInt32,
    unidades_reforestadas_para_restauracion_del_entorno_y_proteccion_del_suelo_y_agua UInt32,
    total_superficie_reforestada_hectareas Float64,
    superficie_reforestada_para_restauracion_del_entorno_y_proteccion_del_suelo_y_agua Float64,
    total_unidades_de_produccion_con_superficie_deforestada UInt32,
    total_superficie_deforestada_hectareas Float64,
    superficie_deforestada_por_causas_naturales Float64,
    unidades_deforestadas_cambio_de_uso_de_suelo_agricola UInt32,
    superficie_deforestada_cambio_de_uso_de_suelo_agricola Float64,
    unidades_deforestadas_cambio_de_uso_de_suelo_pastoreo_de_ganado UInt32,
    superficie_deforestada_cambio_de_uso_de_suelo_pastoreo_de_ganado Float64,
    unidades_deforestadas_cambio_de_uso_de_suelo_habitacional UInt32,
    superficie_deforestada_cambio_de_uso_de_suelo_habitacional Float64,
    unidades_deforestadas_cambio_de_uso_de_suelo_comercio UInt32,
    superficie_deforestada_cambio_de_uso_de_suelo_comercio Float64,
    unidades_deforestadas_cambio_de_uso_de_suelo_industria UInt32,
    superficie_deforestada_cambio_de_uso_de_suelo_industria Float64,
    unidades_deforestadas_cambio_de_uso_de_suelo_servicios UInt32,
    superficie_deforestada_cambio_de_uso_de_suelo_servicios Float64,
    unidades_deforestadas_cambio_de_uso_de_suelo_forestal UInt32,
    superficie_deforestada_cambio_de_uso_de_suelo_forestal Float64,
    unidades_deforestadas_cambio_de_uso_de_suelo_otros_usos UInt32,
    superficie_deforestada_cambio_de_uso_de_suelo_otros_usos Float64
)
    """

### Question:

¿Cuál es la entidad federativa con mayor superficie forestal?
/
Which federative entity has the largest forest area?

In [ ]:
torch.set_float32_matmul_precision('high')

def nl_to_sql(user_input):

    prompt = f'''You are a SQL generator for ClickHouse database.
                - Given a user request in natural language, you will respond with exactly one valid ClikHouse SQL query, nothing else.
                - Use proper table and column names from the schema.
                - Handle aggregations, joins, and filtering appropriately.
                - User natural language questions will be mostly in spanish, less often in English.

                {sql_sintax_differences}

                {schema_info}

                Examples:
                Q: "¿Cuál es la entidad federativa con mayor superficie forestal?"
                A: SELECT
                    entidad_federativa,
                    total_superficie_forestal_ha
                FROM 
                    superficie_bd.superficie_forestal
                ORDER BY
                    total_superficie_forestal_ha
                DESC
                LIMIT 1

                - User request: {user_input}

                - SQL:'''
    
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=150)
    sql_query = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return sql_query.split("SQL:")[-1].strip()

# Example
user_question = "¿Cuál es la entidad federativa con mayor superficie forestal?"
sql = nl_to_sql(user_question)
print("Generated SQL:", sql)

Generated SQL: SELECT entidad_federativa, total_superficie_forestal_ha FROM superficie_bd.superficie_forestal ORDER BY total_superficie_forestal_ha DESC LIMIT 1
```


In [7]:
import re

def extract_sql(text):
    """
    Extracts a clean SQL query from model output.
    Handles:
    - Markdown code fences ```sql ... ```
    - Leading/trailing 'sql' keyword
    - Missing 'SQL:' keyword
    """
    # 1. If the model used "SQL:" somewhere, take only the last part
    if "SQL:" in text or "sql:" in text:
        parts = re.split(r"SQL:|sql:", text, flags=re.IGNORECASE)
        text = parts[-1]

    # 2. Remove Markdown-style code fences
    text = re.sub(r"```.*?```", lambda m: m.group(0).strip("`"), text, flags=re.DOTALL)  # temp remove ticks
    text = re.sub(r"```(?:sql)?", "", text, flags=re.IGNORECASE)
    text = text.replace("```", "")

    # 3. Remove any leading "sql" word or extra spaces/newlines
    text = re.sub(r"^\s*sql\s+", "", text, flags=re.IGNORECASE)

    # 4. Strip whitespace
    text = text.strip()

    return text



# usage:

clean_sql = extract_sql(sql)
print("Cleaned SQL:", clean_sql)

Cleaned SQL: SELECT entidad_federativa, total_superficie_forestal_ha FROM superficie_bd.superficie_forestal ORDER BY total_superficie_forestal_ha DESC LIMIT 1


### Querying database

In [9]:
result = client.query(clean_sql)
result.result_rows

[('Chihuahua', 21610804.96)]

Answer: Chihuahua with 21.6 millions of hectares